In [42]:
import os
import datetime
#import json
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib
#from pandas_datareader import data, wb  # Testing of datareader package
#matplotlib.style.use('ggplot')

# Initial Data Setup
start_date = '2012-05-01'
start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')

In [66]:
data_path = 'C:/Users/Bing/github/longrun/data'
vxv_file = os.path.join(data_path, 'vxv_new.csv')
vxmt_file = os.path.join(data_path, 'vxmt_new.csv')
xiv_file = os.path.join(data_path, 'xiv.csv')
vxx_file = os.path.join(data_path, 'vxx.csv')
svxy_file = os.path.join(data_path, 'svxy.csv')
hvi_file = os.path.join(data_path, 'svxy.csv')
spx_file = os.path.join(data_path, 'spx.csv')
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')
dateparse_dash = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

vxv_df = pd.read_csv(vxv_file, index_col=0, parse_dates=[0], date_parser=dateparse)
vxmt_df = pd.read_csv(vxmt_file, index_col=0, parse_dates=[0], date_parser=dateparse)
vxx_df = pd.read_csv(vxx_file, index_col=0, parse_dates=[0], date_parser=dateparse)
xiv_df = pd.read_csv(xiv_file, index_col=0, parse_dates=[0], date_parser=dateparse)
svxy_df = pd.read_csv(svxy_file, index_col=0, parse_dates=[0], date_parser=dateparse)
hvi_df = pd.read_csv(hvi_file, index_col=0, parse_dates=[0], date_parser=dateparse)
spx_df = pd.read_csv(spx_file, index_col=0, parse_dates=[0], date_parser=dateparse_dash)
xiv = xiv_df[['Adj Close']]
vxx = vxx_df[['Adj Close']]
svxy = svxy_df[['Adj Close']]
hvi = hvi_df[['Adj Close']]
spx = spx_df[['Adj Close']]



In [44]:
# calc ratios and signals
data = pd.merge(vxv_df, vxmt_df, how='left', left_index=True, right_index=True)
data = data.dropna(axis=0, how='any')
# Derived vix futures ratio and moving averages below
ratio = data['VXV'].divide(data['VXMT'])
ratio.sort_index(axis='index', inplace=True)
ratio.name = 'ratio'
ma_60 = pd.Series.rolling(ratio, window=60).mean()
ma_150 = pd.Series.rolling(ratio, window=150).mean()
ma_60.name = 'ma_60'
ma_150.name = 'ma_150'

strat_df = pd.concat([ratio, ma_60, ma_150],  axis=1)
idx = strat_df.isnull().any(axis=1)
strat_df = strat_df[~idx]
strat_df = strat_df[strat_df.index >= start_date]


etf = pd.merge(svxy, xiv, how='left', left_index=True, right_index=True)
etf = pd.merge(etf, vxx, how='left', left_index=True, right_index=True)
etf = pd.merge(etf, hvi, how='left', left_index=True, right_index=True)
etf.columns = ['svxy', 'xiv', 'vxx', 'hvi']


In [45]:
etf.head()

,svxy,xiv,vxx,hvi
Date,,,,
2011-10-04,5.26250,NaN,3382.399902,5.26250
2011-10-05,5.67375,NaN,3216.639893,5.67375
2011-10-06,5.79125,NaN,3160.320068,5.79125
2011-10-07,5.83625,NaN,3214.080078,5.83625
2011-10-10,6.07500,NaN,2983.040039,6.07500


In [46]:
# First strat df with all etf data and ratio
#longrun = pd.merge(etf, strat_df , how='left', left_index=True, right_index=True)

# Strategy signals can be calculated independent of underlying data first, merge signals with price data later
# 100% Long signal
strat_df['signal'] = np.where((strat_df['ma_60']<1) & (strat_df['ma_150']<1) & (strat_df['ratio'] < strat_df['ma_60']) & (strat_df['ratio'] < strat_df['ma_150']), 1, 0)

# 50% long signal
strat_df['signal_50'] = np.where((strat_df['ma_60']<1) & (strat_df['ma_150']<1) & (strat_df['ratio'] > strat_df['ma_60']) & (strat_df['ratio'] < strat_df['ma_150']), 1, 0)

# Get enter/out signal
idx = np.where(strat_df.signal[1:].values != strat_df.signal[:-1].values)[0] + 1
idx_50 = np.where(strat_df.signal_50[1:].values != strat_df.signal_50[:-1].values)[0] + 1

# Get buy/sell cells for 100% Long
if strat_df.signal[0] == 0:
    buy_idx = idx[::2]
    sell_idx = idx[1::2]
else:
    buy_idx = np.insert(idx[1::2], 0, 0)
    sell_idx = idx[::2]

strat_df['bs'] = ''
strat_df.bs[buy_idx] = 'buy'
strat_df.bs[sell_idx] = 'sell'

# Get buy/sell cells for 50% Long
if strat_df.signal_50[0] == 0:
    buy_50_idx = idx_50[::2]
    sell_50_idx = idx_50[1::2]
else:
    buy_50_idx = np.insert(idx_50[1::2], 0, 0)
    sell_50_idx = idx_50[::2]

strat_df['bs_50'] = ''
strat_df.bs_50[buy_50_idx] = 'buy'
strat_df.bs_50[sell_50_idx] = 'sell'

# Output strat_df to csv
strat_df.to_csv(os.path.join(data_path, 'strat_signal.csv'))



C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:37: SettingWithCopyWarning

In [47]:
# Start to build the longrun strat portfolio with underlying stock/etf
# First merge stock with strat_df
all_data = pd.merge(etf, strat_df , how='left', left_index=True, right_index=True)
longrun = all_data[all_data.index >= start_date]
# etf_name = 'xiv'
etf_name = 'svxy'


In [48]:
# Now shift the underlying price (XIV) backward 1 day to avoid look ahead bias
longrun[etf_name] = longrun[etf_name].shift(-1)
longrun[etf_name].fillna(method='ffill', inplace=True)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda2\envs\python3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [49]:
# Below are the trade/rebalance logics for the portfolio strating from trade day
# Start to build the strategy portfolio
capital = 100
n_shares = 0
capital_prev = capital
n_prev = n_shares

port = []
n_stock = []
cap_list = []
cash_list = []
# Below are the trade/rebalance logics for the portfolio strating from trade day
for i in np.arange(0, len(longrun.index)):
    price = longrun[etf_name].iloc[i]
    signal = longrun.signal[i]
    bs = longrun.bs[i]
    # update today's share and capital from yesterday
    n = n_prev
    capital = capital_prev
    # if today is buy
    if signal == 1 and bs == 'buy':
        n = capital/price
    # if today is hold
    elif signal == 1 and bs == '':
        capital = n * price
    # if today is sell/close
    elif signal == 0 and bs == 'sell':
        capital = n * price
        n = 0

    n_prev = n
    capital_prev = capital
    port.append(capital)
    n_stock.append(n)

In [50]:
port_df = longrun[[etf_name]]
port_df['strat'] = port
port_df['n'] = n_stock


C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [51]:
import matplotlib.pyplot as plt
import matplotlib
from pandas_datareader import data, wb  # Testing of datareader package
matplotlib.style.use('ggplot')

In [52]:
%matplotlib inline  

In [62]:
def get_port_summary(data):
    daily_ret = data.pct_change(1)[1:]
    daily_std = np.nanstd(daily_ret)
    annual_std = daily_std * np.sqrt(252)
    total_return = data[-1]/data[0] - 1
    sharpe_ratio = np.sqrt(252) * np.average(daily_ret) / daily_std

    ndays = (data.index[-1] - data.index[0]).days
    nyears = ndays/252
    year_return = np.power(1 + total_return, 1/nyears) - 1
    return {'total_return': total_return, 'yearly_return': year_return, 'yearly_std': annual_std, 'sharpe_ratio': sharpe_ratio}




In [64]:
perf = get_port_summary(port_df['strat'])
perf

{'sharpe_ratio': 1.1381742754876161,
 'total_return': 4.6072852277569005,
 'yearly_return': 0.2529011179043108,
 'yearly_std': 0.33918378139148442}